In [8]:
from keras.models import load_model, Sequential
from keras.layers import Conv2D, Input
from keras.layers import LocallyConnected2D
from keras.optimizers import Adam
import numpy as np

In [9]:
def cnn_layer_to_fcn_layer(cnn_layer):
    conv_weights, conv_biases = cnn_layer.get_weights()
    layer_config = cnn_layer.get_config()
    fcn_layer = LocallyConnected2D(
        filters=layer_config['filters'],
        kernel_size=layer_config['kernel_size'],
        strides=layer_config['strides'],
        padding=layer_config['padding'], 
        activation=layer_config['activation']
    )
    output_shape = cnn_layer.output_shape
    input_shape = cnn_layer.input_shape
    fcn_weights = np.empty(
        (
            output_shape[1] * output_shape[2],
            input_shape[3] * layer_config['kernel_size'][0] * layer_config['kernel_size'][1],
            layer_config['filters']
        )
    )
    fcn_biases = np.empty(
        (
            output_shape[1],
            output_shape[2],
            layer_config['filters']
        )
    )
    for i in range(fcn_weights.shape[0]):
        fcn_weights[i,:,:] = conv_weights.reshape((fcn_weights.shape[1],fcn_weights.shape[2]))
    for i in range(fcn_biases.shape[0]):
        for j in range(fcn_biases.shape[1]):
            fcn_biases[i,j,:] = conv_biases
    return fcn_layer, [fcn_weights, fcn_biases]

In [10]:
CNN_FILENAME = "SavedModels/model.h5"

In [11]:
pretrained_cnn = load_model(CNN_FILENAME)
pretrained_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        4864      
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 5, 128)         204928    
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 256)         295168    
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 1024)              2360320   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 2,875,530
Trainable params: 2,875,530
Non-

In [12]:
fcn = Sequential()
fcn.add(Input(shape=(32, 32, 3)))
for layer in pretrained_cnn.layers:
    if isinstance(layer, Conv2D):
        new_layer, fcn_weights_biases = cnn_layer_to_fcn_layer(layer)
        fcn.add(new_layer)
        new_layer.set_weights(fcn_weights_biases)
    else:
        fcn.add(layer)
fcn.build()
original_optimizer = pretrained_cnn.optimizer
fcn.compile(
    type(original_optimizer).from_config(original_optimizer.get_config()),
    pretrained_cnn.loss,
    metrics=['accuracy']
)
fcn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 locally_connected2d_3 (Loca  (None, 14, 14, 64)       953344    
 llyConnected2D)                                                 
                                                                 
 locally_connected2d_4 (Loca  (None, 5, 5, 128)        5123200   
 llyConnected2D)                                                 
                                                                 
 locally_connected2d_5 (Loca  (None, 3, 3, 256)        2656512   
 llyConnected2D)                                                 
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 1024)              2360320   
                                                      

In [13]:
fcn.save('SavedModels/fcn.h5')